In [ ]:
# 데이터 준비
from google.colab import files
uploaded = files.upload()

#파일 선택 눌러서 원하는 파일 가져오기 가능

# colab > run_project 폴더에 있는 colab_train_test.zip 파일을 선택합니다.
my_data = 'colab_train_test.zip'
!unzip colab_train_test.zip

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

# 많이 사용하는 함수/ 디렉토리에서 이미지를 가지고오면서 증강. 하나하나 읽어온 후에 증강하는 대신에  (더 많이 사용하는 코드)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, initializers, regularizers, metrics

In [ ]:
np.random.seed(3)
tf.random.set_seed(3)

# 이미지 데이터 생성기 만든 것
# 데이터 읽어오는 즉시 파라미터들 적용. 한장 읽어오는 과정에서 여러장이 된다
train_datagen = ImageDataGenerator(rescale=1./255,
                                  horizontal_flip=True,     #수평 대칭 이미지를 50% 확률로 만들어 추가합니다.
                                  width_shift_range=0.1,  #전체 크기의 10% 범위에서 좌우로 이동합니다.
                                  height_shift_range=0.1, #마찬가지로 위, 아래로 이동합니다.
                                  #rotation_range=5,
                                  #shear_range=0.7,
                                  #zoom_range=[0.9, 2.2],
                                  #vertical_flip=True,     >> 이미지가 완전 대칭인 경우에는 의미 딱히 없어짐...
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(    # train_datagen을 거쳐서 가져오게 되는것
       'train',   #학습셋이 있는 폴더의 위치입니다. 디렉토리 이름
       target_size=(180, 180),
       batch_size=5,
       color_mode="rgb",
       class_mode='binary')

# 테스트 셋은 이미지 부풀리기 과정을 진행하지 않습니다.
# traingenerator 썼으면 test도 test_gen 똑같이 해줘야 한다. 그렇지만 test는 이미지 증강 안한다!!! 그래도 똑같이 해줘야한다는 것!!!
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
       'test',   #테스트셋이 있는 폴더의 위치입니다.
       target_size=(180, 180),
       batch_size=5,
       color_mode="rgb",
       class_mode='binary')

In [ ]:
class_map = {
    0: 'AD',
    1: 'Normal',
}

In [ ]:
for x, y in train_generator:
    print(x.shape, y.shape)
    print(y[0])

    fig, axes = plt.subplots(1, 5)
    fig.set_size_inches(15, 15)
    for i in range(5):
        axes[i].imshow(x[i])
        axes[i].set_title(class_map[int(y[i])], fontsize=15)
        axes[i].axis('off')
    plt.show()
    break

# 모델 디자인

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers


conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",   #imagenet으로 학습한 가중치 사용
    include_top=False,    # 뒤파트를 top이라고 함. 뒤파트는 안가지고 오겠다는 뜻. 앞의 가중치만 쓰겠다.
    input_shape=(180, 180, 3))    # 여기까지 신경망 base모델
conv_base.summary() #모델 구조. 뒤부분 없어서 flatten층 dense층 .. 잘라서 없음 잘린 형태


#출력층 있어야하므로 flatten부터 붙이는것. 이진 분류니까 sigmoid
####모델
inputs = keras.Input(shape=(180, 180, 3))
x = conv_base(inputs)  # VGG16을 통한 특징 추출

x = layers.Conv2D(64, (3, 3), activation='relu')(x)  # Conv2D 추가
x = layers.Conv2D(32, (3, 3), activation='relu')(x)

# 마지막 Flatten, Dense 추가
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(2, activation='softmax')(x)  # 이진 분류


model = keras.Model(inputs, x)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics=['accuracy'])  # binary_crossentropy 해도 될것/ learning_rate 디폴트는 0.0001

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Define model checkpoint to save the best model
model_checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)


# 모델을 실행합니다
history = model.fit(
       train_generator,

       steps_per_epoch=20,  #적절한 값으로 조절
       epochs=20,   #적절한 값으로 조절
       validation_data=test_generator,
       validation_steps=10,
       callbacks=[early_stopping, model_checkpoint] # Add callbacks to the fit function
)


In [ ]:
# 성능 확인하기
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(train_generator, steps=len(train_generator))
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Training Loss: {train_loss:.4f}")

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

# LIME

In [ ]:
# lime 진행

!pip install lime

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

In [ ]:
# 예측 함수 정의
def predict(input_img):
    input_img = preprocess_input(input_img)
    return model.predict(input_img)

In [ ]:
from lime import lime_image

# test_generator에서 배치(batch)를 하나 가져옵니다.
batch_images, batch_labels = next(test_generator)

# 첫 번째 이미지에 대해 LIME을 적용
img = batch_images[0]  # 첫 번째 이미지를 선택

# 이미지 전처리 (필요한 전처리 작업: 0~1 범위로 스케일링)
img_array = img.astype('double')

# LIME 설명자 생성
explainer = lime_image.LimeImageExplainer()

# LIME 해석 수행
explanation = explainer.explain_instance(img_array,
                                         predict,  # 예측 함수
                                         top_labels=5,
                                         hide_color=0,  # 배경을 검정색으로
                                         num_samples=200)

# 특정 클래스에 대한 설명 시각화 (예측 상위 클래스 사용)
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
    # temp는 LIME 설명 이미지, mask는 강조된 영역입니다.


In [ ]:
# 결과 시각화
plt.imshow(mark_boundaries(temp, mask))
plt.title("LIME Explanation for Grayscale Image")
plt.axis('off')
plt.show()
